# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is Classification problem. As it is to predict a class label, which is a choice from a predefined list of possibilites(discrete value), in this case need to find whether student needs intervention or not. This is Binary classification not the Multiclass classification.

If this is to find the percentage of intervention needed for each student, then as it is to predict a continuous value, it is Regression problem.  

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [4]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
%matplotlib inline
# Read student data
student_data = pd.read_csv("student-data.csv")

print("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [5]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = len(student_data[student_data.passed == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data.passed == 'no'])

# TODO: Calculate graduation rate
grad_rate = 100 * n_passed / (n_passed + n_failed)

# Print the results
print("Total number of students: {}".format(n_students))
print("Number of features: {}".format(n_features))
print("Number of students who passed: {}".format(n_passed))
print("Number of students who failed: {}".format(n_failed))
print("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [6]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print("Feature columns:\n{}".format(feature_cols))
print("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print("\nFeature values:")
print(X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [7]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [8]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train
 
testsize = (num_test/(num_test+num_train))  
# TODO: Shuffle and split the dataset into the number of training and testing points above

X_train_tmp, X_test_tmp, y_train_tmp, y_test_tmp = train_test_split(X_all, y_all, test_size=testsize, random_state=42)
X_train = X_train_tmp
X_test = X_test_tmp
y_train = y_train_tmp
y_test = y_test_tmp

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
GradientBoostingClassifier
GradientBoostingClassifier(add new models to the ensemble sequentially) VS Random Forest (simple averaging of models)
Real-World ex:
Spam Filtering
https://www.microsoft.com/en-us/research/publication/asymmetric-gradient-boosting-with-application-to-spam-filtering/
Classify Human Activities
https://sasglobalforum2016.lanyonevents.com/connect/sessionDetail.ww?SESSION_ID=11801
Strengths:
Better learners than Random Forests when well-tuned
Add new trees that compliments the already built ones
Can be distributed and very fast
Are derived by optimized a objective function
Automatically detects(non-linear) feature interaction
Weaknesses:
Training takes longer , because of the fact that trees are built sequentially
Over fitting if the data is noisy  
Reason to choose:
https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3885826/
combines a set of weak learners and delivers improved prediction accuracy.
each particular iteration, a new weak, base-learner model is trained with respect to the error of the whole ensemble learnt so far. So loss function to be optimized. Generic enough that any differentiable loss function can be used

References: 
https://www.quora.com/What-are-the-advantages-disadvantages-of-using-Gradient-Boosting-over-Random-Forests
https://www.quora.com/When-would-one-use-Random-Forests-over-Gradient-Boosted-Machines-GBMs


Support Vector Mahcines(SVM) 
Real-World ex:
widely applied in the biological and other sciences, like classify proteins
Predict the calss of a tumour from gene expression data
Text and hypertext categorization
Classification of images
Hand-written characters
https://en.wikipedia.org/wiki/Support_vector_machine  
 
Strengths:
Optimal hyperplane for linearly separable patterns
http://www.svms.org/tutorials/Berwick2003.pdf
Maximize the margin around the separating hyperplane.
Extend to patterns that are not linearly separable by transformations of original data to map into new space – Kernel function
SVM algorithm for pattern recognition  
has a regularisation parameter, which makes the user think about avoiding over-fitting
uses the kernel trick, so you can build in expert knowledge about the problem via engineering the kernel
Weaknesses:
moves the problem of over-fitting from optimising the parameters to model selection.
kernel models can be quite sensitive to over-fitting the model selection criterion
inefficient to train
Reason to choose:
if the problem is non-linear it can perform and then optimize with regularisation parameters and kernel methods

References: 
http://stats.stackexchange.com/questions/24437/advantages-and-disadvantages-of-svm
https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms

Stochastic Gradient Descent Classifier
Simple yet very efficient approach to discriminative learning of linear classifiers under convex loss functions such as (linear) Support Vector Machines and Logistic Regression. 
Real-World ex:
large-scale and sparse machine learning problems often encountered in text classification and natural language processing.

Strengths:
Efficiency.
Ease of implementation (lots of opportunities for code tuning).

Weaknesses:
SGD requires a number of hyperparameters such as the regularization parameter and the number of iterations.
SGD is sensitive to feature scaling.

Reason to choose:
if the problem is linear it's computationally efficient

References: 
http://scikit-learn.org/stable/modules/sgd.html

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [9]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))
    print("****")

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [10]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.linear_model import SGDClassifier 
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier  
# from skearln import model_C
random_state = 29
# TODO: Initialize the three models
 
clf_A = GradientBoostingClassifier(n_estimators=500,random_state=random_state,max_depth=16,learning_rate=.1,subsample=.5) 
clf_B = SVC(kernel='rbf', random_state=random_state, gamma=0.10, C=10.0)
clf_C = SGDClassifier(n_iter=50,shuffle=True,loss="log", \
                learning_rate = "constant",eta0=0.0001,fit_intercept=True, penalty="none",random_state=random_state)
classifiers = [clf_A, clf_B, clf_C]
# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

train_sets = [(X_train_100, y_train_100),(X_train_200, y_train_200),(X_train_300, y_train_300)]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
for clf in classifiers:
    for X, y in train_sets:
        train_predict(clf, X, y, X_test, y_test)

Training a GradientBoostingClassifier using a training set size of 100. . .
Trained model in 0.6220 seconds
Made predictions in 0.0123 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7519.
****
Training a GradientBoostingClassifier using a training set size of 200. . .
Trained model in 0.7956 seconds
Made predictions in 0.0040 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0015 seconds.
F1 score for test set: 0.8148.
****
Training a GradientBoostingClassifier using a training set size of 300. . .
Trained model in 1.4338 seconds
Made predictions in 0.0050 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.8235.
****
Training a SVC using a training set size of 100. . .
Trained model in 0.0060 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7600.
****
Training a SVC usin

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** GradientBoostingClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.4698|                     0   |             1     |  0.759               | 
| 200               |        0.7925          |   0                     |     1             |  0.8148               |
| 300               |             1.2714            |       0.0146                 |      1            |      0.8235    |

** SVC **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0015  |         0.0010               |         1        |           0.7600      |
| 200               |     0.0045             |  0.0025                     |  1               |  0.7733               |
| 300               |          0.0090               |    0.0020                    |    1              |    0.7947     |

** SGDClassifier **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0156                      |     0                   | 0.7843                 | 0.7838                |
| 200               |                0.0156         |                 0     |      0.8155              |     0.7742            |
| 300               |                  0.0156       |          0              |           0.8162       |          0.7815       |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
Out of 3 - can choose GradientBoostingClassifier as the best one.
It an be distributed and very fast and derived by optimized a objective function , it's extensible as new models that compliments the already built ones.

This has the best F1 Score - it may have huge diffence between Training and testing F1 Score, but can be optimized and add new model that compliments the existing ones. Prediction time for testing data is high compare to other as it's distributable it can be faster than the others.

Cost is not going to be huge as it has less data, even we keep increasing the data it is going to be performed better than the others.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **
Imagine  20 teams (trees). A boss at the top, then subordinates, then more  subordinates, and so on. Team members are explanatory variables. Assume,  Trees = 20 and Depth (number of members in each team) = 5. So each team  will have 5 members, and total members = 100. We give them a book to  read, and then they will have to answer 20 questions (Number of  observations in our data). Assume they have binary answers: Yes or No.  Now, we start the process. The aim of the process is to maximum correct  answers by building 20 teams having 5 members each. Any member can be a  part of more than 1 team, and any member can have more than 1 more than 1  role in same team. The member which have maximum roles is the most  important variable of our model.

The  process starts with a random guess of answers. Then it calculates error  ( = Actual - Predicted Answer). Next step, it build a team of 5  members, which reduces the error by maximum. Again, it calculates the  error. The second team (tree) has to reduce it further. But next team  doesn't trust its previous partner fully, so it assume that answers are  correct with x probability (learning rate). This process go on till 20  teams are build. So in the process, we have to decide, how many teams to  build (trees), members in each team(depth) and learning team, so that  error in the end is minimum. This can only be done by trial and error  method.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [16]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer 
print(1)
# TODO: Create the parameters list you wish to tune
parameters = [{'max_depth':[1,2,3], 'n_estimators':[10,20,30],'max_features':['sqrt', 'log2', 1]}
              ,{'max_depth':[1,2,3], 'subsample':[1,0.8,0.3]}]
 
# TODO: Initialize the classifier
clf = GradientBoostingClassifier(learning_rate=0.5, min_samples_split=5,min_samples_leaf=2,
                                 max_depth=8,max_features='sqrt',subsample=0.5,random_state=random_state)
def performance_metric(label, prediction):
    return f1_score(label, prediction, pos_label='yes')
# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(performance_metric,greater_is_better=True)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, param_grid = parameters, scoring =f1_scorer, cv=5, verbose=0)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print("Best score on hold-out data during grid search: {:.4f}".format(grid_obj.best_score_))
# Report the final F1 score for training and testing after parameter tuning
print("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))

1
Best score on hold-out data during grid search: 0.8101
Made predictions in 0.0010 seconds.
Tuned model has a training F1 score of 0.8348.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.8085.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
For Training F1 Score - 83.48%, for Testing 80.85%.
Where as untuned model - F1 81.48% and 82.35% respectively.
When compared to untuned model training score improved, where as testing score is corrected. So tuning helped us to improve test accuracy and avoid overfitting 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.